In [1]:
# plots
%matplotlib inline

import matplotlib.pyplot as plt
from pylab import rcParams
import math
rcParams['figure.figsize'] = 15, 8

import math
import scipy

import json

# Pandas
import pandas as pd
from pandas import DataFrame

# math api
import numpy as np

# time lib
import time
import datetime as dt

NOW = dt.datetime.now()


import seaborn as sns
sns.set()




In [2]:
flat_data = pd.read_csv("../ressources/data/financial_sells_100000.csv")
flat_data = flat_data.fillna(0).replace('-', np.NaN)
flat_data = flat_data.drop(columns=['Unnamed: 0'])
flat_data.head()

,company_name,company_id,country,sector_name,order_id,product_name,NBI,order_date
0,Daniel and Sons,1,Espagne,Electrical industry,1,MCIB Financing,-6.552610,2016-10-27
1,"Carter, Hernandez and Thompson",2,United Kinkdom,Chemical Industry,2,Equity,83.814604,2016-07-30
2,Martinez-Davis,3,United State,Energy industry,3,Commodity Financing,79.246489,2017-11-12
3,Bradley-Burns,4,Italy,Metallurgical industry,4,Mergers and Acquisition,111.870450,2015-09-14
4,Bradley-Burns,4,Italy,Metallurgical industry,5,Liquidity Management,88.378200,2017-09-11


# first implementation of Cross-Selling

In [9]:
dict_combinaison = dict()

for sector in flat_data['sector_name'].unique(): 
    for country in flat_data['country'].unique():
        
        
        query_fetch = 'country == "{}" and sector_name == "{}"'.format(country, sector)
        result_query = flat_data.query(query_fetch)
        all_products = result_query['product_name'].unique()
        
        for company_name in result_query['company_name'].unique():
            bought_products = set(flat_data.query('company_name == "{}"'.format(company_name))['product_name'].unique())
            all_products = set(all_products)
            unbought_products = all_products.symmetric_difference(bought_products)
            
            if( len(unbought_products) > 0 ):
                dict_combinaison[ len(dict_combinaison) ] = {
                    'company_name': company_name,
                    'sector': sector,
                    'country': str(country),
                    'bought_product': str(bought_products),
                    'unbought_product': str(unbought_products),
                    'diff': len(unbought_products)
                }



print(dict_combinaison.get(0))
print(len(dict_combinaison))
flat_data.query(' company_name == "Powers-Woodard" and country == "Espagne" ')

{'company_name': 'Powers-Woodard', 'sector': 'Electrical industry', 'country': 'Espagne', 'bought_product': "{'Commodity Financing'}", 'unbought_product': "{'Forex', 'MCIB Financing', 'Fund Administration', 'Financing'}", 'diff': 4}
13


,company_name,company_id,country,sector_name,order_id,product_name,NBI,order_date
7055,Powers-Woodard,71,Espagne,Electrical industry,7056,Commodity Financing,6.557642,2016-01-27


In [136]:
result_df = pd.DataFrame()
for value in dict_combinaison.values():
    result_df = result_df.append(value, ignore_index=True)
    
result_df.to_csv('../ressources/data/2_cross_selling_result.csv', sep=';')


# second implementation of cross_selling

In [11]:
def apply_cross_selling(_df):
    # count all unique clients
    count_distinct_customers = _df['company_name'].nunique()

    result_cross_selling = dict()

    # for each item, compare with otel items
    for product_1 in _df['product_name'].unique(): 
        
        #print( product_1 )
        
        #count all users who bougth this item1
        query_product_1 = "product_name == '{0}'".format(product_1)
        result_query_1 = _df.query(query_product_1)


        for product_2 in _df['product_name'].unique():
            dict_key1 = (product_2, product_1)
            dict_key2 = (product_1, product_2)
            score = 0

            if ( dict_key1 in result_cross_selling ):
                #print('already exists', end=' - ')
                continue

            #count all users who bougth this item1
            query_product_2 = "product_name == '{0}'".format(product_2)
            result_query_2 = _df.query(query_product_2)

            list_customer_product_1 = set( result_query_1['company_name'].unique() )
            list_customer_product_2 = set( result_query_2['company_name'].unique() )
            common_customers = list_customer_product_1.intersection(list_customer_product_2)

            score = len( common_customers ) / count_distinct_customers

            

            result_cross_selling[ dict_key1 ] = score
            result_cross_selling[ dict_key2 ] = score
            

apply_cross_selling(flat_data)

Liquidity Management
Credit
Loans syndication
Equity
Custody Services
Export Finance
Asset Servicing
ALD Car Renting and Fleet Management
Mergers and Acquisition
International Retail Banking
Commodity Financing
Asset Based Financing
Asset Management
Fund Distribution Services
Prime & Clearing
Life and Dammage insurance
Clearing Services
Commodities
Leverage Finance
Fund Administration
MCIB Financing
Issuer Services
General Purpose Financing
Hybrids
Others
Financing
Forex
